# Lab 5: Running and Benchmarking your 5G Network

In this lab, you will use the configuration files you adapted in the previous lab to run your very own end-to-end 5G SA network. You'll be challenged to configure your OpenAirInterface (OAI) 5G network as closely as possible to your originally planned network from lab 3. You will use the OAI docker images to spin up containerized parts of your 5G network. Then, once you have a fully functional network, you will capture various packet exchanges between the different network components. Finally, you'll finish this lab by performing some basic benchmarks.

## Prerequisites

### Hardware Requirements

OAI can be quite resource intensive depending on what you do with it. For what you will be doing, it shouldn't be a problem to run the whole OAI 5G network on a single laptop, providing that you have a relatively modern system.

However, there is one important thing you should keep in mind. The OAI gNB and UE implementations make heavy use of the AVX2 instruction set. Therefore, you should make sure that your CPU implements this instruction set, or otherwise you will not be able to run these components. In case your CPU does not have support for AVX2 (which is very unlikely, unless you have an Apple Silicon MacBook or a laptop that is over a decade old), you can use a physical node on the [Virtual Wall](https://doc.ilabt.imec.be/ilabt/virtualwall/), which you can access using jFed. In fact, I have tested all of the OAI-related exercises on a pcgen6 node on the Virtual Wall using Ubuntu 20.04 (see image), so if you plan on using it at some point, this setup should not cause any issues.

![Accessing physical nodes from the Virtual Wall using jFed](images/jfed_virtual_wall.png)

### Install Docker

Since you will use the dockerized version of OAI, you should install Docker on the machine that you plan to run it on. 

In case you don't have Docker installed yet, follow the installation instructions from the Docker [documentation](https://docs.docker.com/engine/install/).

> <span style="color:green">**Hint**</span>: after installing Docker, add your user to the docker group and activate the changes, so you don't have to run the docker commands with `sudo`.
> ```bash
> sudo usermod -aG docker $USER
> newgrp docker
> ```

### Setting up Wireshark 

You will use Wireshark to capture packets that are sent between the different components of your 5G network.

By default, Wireshark will not be able to correctly dissect and display packets from certain 5G protocols. In 5G radio networks, Radio Link Control (RLC) and Radio Resource Control (RRC) messages are typically exchanged between the UE and gNB using User Datagram Protocol (UDP) as a transport protocol. In fact, if you were to look at an OAI packet capture between the UE and gNB (see later) with the default Wireshark profile, you would see a bunch of UDP packets with unreadable datagrams. Therefore, you will need to do some configuring inside your Wireshark.

Before messing with your Wireshark settings and protocol configurations, I recommend you create a new configuration profile by going to `Edit > Configuration Profiles`. There, you can create and activate a new profile (e.g., "5G-NR") where all of your configurations will be saved. This will allow you to revert back to your default profile when you are done with the labs.

Next, you can start configuring the way Wireshark dissects RLC and RRC packet captures by following the OAI [documentation](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/common/utils/T/DOC/T/wireshark.md#configure-wireshark-for-nr).

## *The OAI Games*: Configuring and Debugging Your 5G Network

Before you start delving into packet captures and benchmarks, you will first have to make sure that you are able to run the network using your own configuration files. There is a big chance that your network will not "just work" when you run it for the first time, either because you misconfigured something, or because OAI is quite picky about particular combinations of parameters. In fact, it can be tricky to correctly configure a working OAI system. The 5G NR specification is very flexible by design, which makes initial setup more complex due to the huge amount of possible configuration options. 

Configuring and running an OAI system without errors can be complex due to the flexibility and multitude of configuration options provided by the 5G NR specification. On top of this, OAI can be quite picky about particular combinations of parameters, which creates additional challenges. To make the time you will spend on configuring your OAI system worth your while, we introduce: *The OAI Games*.

### Objective

The main objective of *The OAI Games* is to configure your OAI 5G network to closely resemble your initial network design from the previous lab sessions. The group that achieves the closest alignment between their configured network and their original plan will also receive one extra point for the 5G labs.

### Expectation and Grading

Unlike previous exercises, this exercise is intentionally open-ended. This exercise is designed to test your ability to configure the system independently, as well as your overall understanding of 5G concepts. Therefore, for this exercise, you will be evaluated based on two main criteria:

1. Your ability to keep your resulting configuration file as close as possible to your original file from the previous lab; **and**
2. Your ability to document your modifications and clearly explain your debugging process.

Therefore, even if you did not manage to make your system work with your original configuration, you will still be rewarded if you can clearly document the changes you've made throughout your debugging process.

### Rules of the Game
The rules of the game are as follows:
- You are allowed to change any files you want.
- You are allowed to revert back to some of the default configuration parameters, as long as you clearly document why you did that you did.

> <span style="color:green">**Hint**</span>: If you need a reference for a fully operational network, you can use the provided configuration files in their default state. However, keep in mind that simply using the default network for the rest of the lab without modifications will result in point deductions!

In the following two subsections, you'll briefly be guided on how to run the different network components, and you'll be given some space to document your configuration and debugging process.

### How to Run

To run your 5G network, you will use the resources available in the `oai` folder, where the configuration files you changed in the previous lab are stored. The `oai-cn5g`, `oai-gnb`, and `oai-ue` subfolders each contain a Docker Compose file to run a specific part of the network (CN, gNB, and UE, resp.).

Always keep in mind the following deployment order when running the Docker Compose files: CN $\rightarrow$ gNB $\rightarrow$ UE. Deploying the containers in any other order will most likely result in issues, so make sure that the necessary services are running before moving on.

**1. OAI Core Network (CN)**

Go to the `oai-cn5g` folder and run 

```shell
docker compose up -d
```

The `-d` flag will make sure that the services are started in the background. Before moving on to the deployment of the gNB, make sure that all the CN functions are started correctly by running `docker compose ps -a`. If you configured everything properly, you should eventually see a `(healthy)` status on all the services.

**2. OAI gNodeB (gNB)**

Once all the CN functions are up and running (and marked healthy), you should be able to go to the `oai-gnb` folder and setup the gNB using

```shell
docker compose up
```

I recommend you run the gNB without the `-d` flag in a separate terminal window, since 1) the compose file only has one service, which makes reading the output of the command easier; and 2) the container logs will be useful to see whether the UE is able to connect to the gNB. 

**3. OAI User Equipment (UE)**

Once the CN is running and correctly recognizes your running gNB, you are ready to start the UE. Go to the `oai-ue` folder and run

```shell
docker compose up
```

Just like the gNB, it is a good idea to run the UE in a separate terminal window to keep an eye on the logs.

**4. Connectivity test**

If you got to this point without too many errors, you should now have a fully functioning 5G SA network. In order to verify this, you can run a ping from the UE to the external network (in this case, the `oai-ext-dn` container). Make sure you ping using the correct interface!

**Question**: Run a packet capture on the `demo-oai` interface, and ping the `oai-ext-dn` container from the UE over the tunnel interface. You should notice something different about your captured ICMP packets. What do you notice and why is this the case?

**Answer**: <span style="color:red">**TODO**</span>

### Debugging

**Question**: Document here what you did to get an operational OAI network. Be clear and thorough!

**Answer**: <span style="color:red">**TODO**</span>

## Packet Traces

Once your custom network is running, you are ready to start capturing some of the interactions between the different network components. **Make sure you run everything below this point using your own network!** The packet captures will reflect some of the configurations you did above, and will tell us whether you configured things correctly.

For most of the captures, you can use the [5G SA access procedure](https://www.eventhelix.com/5g/standalone-access-registration/5g-standalone-access-registration.pdf) as a reference.

### gNB Registration Procedure

Start by capturing the gNB registration process that occurs between the CN and a gNB that joins the network. To capture the necessary packets, follow the steps below:

1. Run the CN,
2. Start a packet capture on the network interface created by Docker,
3. Launch the gNB until it is fully connected.

**Question**: Take a look at the logs of the AMF. Provide the part of the logs that shows that the gNB is recognized by the core. 

**Answer**: <span style="color:red">**TODO**</span>

**Question**: Which two packets reflect the gNB registration process, and what protocol do they use? Export them into a capture file called [lab5-gnb-setup.pcap](traces/lab5-gnb-setup.pcap), and briefly explain their `Item` fields.

**Answer**: <span style="color:red">**TODO**</span>

### UE RRC Connection Setup Procedure

Next, you'll capture the message exchange between the UE and gNB that occurs right after the random access procedure. 

Note: In order to capture lower layer packets (RLC, RRC) between the UE and gNB, you will have to enable packet capturing from within the gNB itself (as explained in the [documentation](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/openair2/UTIL/OPT/README.txt)). You won't be able to see them if you simply capture the packets on the Docker network interface without enabling packet capturing at the gNB!

**Question**: Take a look at the logs of the AMF and show the part of the logs that proves the UE is recognized by the core. 

**Answer**: <span style="color:red">**TODO**</span>

**Question**: Capture the RRC connection setup procedure between the UE and the gNB in a file called [lab5-rrc-setup.pcap](traces/lab5-rrc-setup.pcap). What is the purpose of this procedure? Briefly explain what each packet contains.

**Answer**: <span style="color:red">**TODO**</span>

### Access Registration Procedure

Once the UE has established a connection with the gNB, the gNB will forward the UE's registration request to the CN. Then, the CN functions will exchange messages to authenticate the UE, create a session, and establish a network connection.

For simplicity, you will capture the whole access procedure in one pcap file called [lab5-access-registration.pcap](traces/lab5-access-registration.pcap). Throughout the following questions, you'll be asked to refer to specific packets in this pcap file to explain various procedures. In order to create this pcap file, you should perform the following steps:

1. Run the CN,
2. Start a packet capture on the network interface created by Docker,
3. Run the gNB until it is fully connected,
4. Run the UE until it is fully connected.

Use the filter below when capturing packets (assuming you haven't changed any ports in your configuration), either using `tcpdump` or Wireshark. The filter allows only the following packets to be captured: control plane RAN (SCTP), ICMP, HTTP of control in CN, PFCP, MySQL
```
sctp or icmp or port 80 or port 2152 or port 3306 or port 8080 or port 8805 or port 9090 or port 33060 or port 38412
```

**Question**: Create a sequence diagram of the message exchange that you captured between gNB and CN functions during the access procedure, from the `InitialUEMessage` message to the `InitialContextSetupResponse` message. For each exchanged message, give:

1) the name of the message;
2) the protocol used;
3) a short description of what the message does;
4) a packet ID, referring to the ID of the packet in your capture file corresponding to the message.

Make sure you capture at least:
1) The `InitialUEMessage`
2) Authentication & security procedure
5) UPF setup procedure
7) `InitialContextSetupRequest` message

*(see [5G SA access procedure](https://www.eventhelix.com/5g/standalone-access-registration/5g-standalone-access-registration.pdf). Note that the procedure you'll capture won't exactly match the procedure listed in this document, since your core is a bit simpler)*

**Answer**: <span style="color:red">**TODO**</span>

## Benchmarking

Finally, you will finish this lab by doing some basic benchmarking of your 5G network. To do so, you will run both a downlink and uplink iperf session to evaluate the end-to-end throughput of your system.

### Downlink Benchmark

To perform a downlink benchmark, you will start an iperf server in the UE container and start an iperf client in the `oai-ext-dn` container (i.e., the "external network") towards the server.

In the UE container:
```shell
iperf -s -B <ip_of_tunnel_interface> -u -i 1
```

In the `oai-ext-dn` container:
```shell
iperf -c <ip_of_ue> -u -i 1 -t 20 -b <amount_of_data_to_send> # you might need to play around with the -b option a bit
```

**Question**: List the commands that you ended up running, and give the average downlink throughput. Does this value align with your expectations, and why (not)?

**Answer**: <span style="color:red">**TODO**</span>

### Uplink Benchmark

Similarly, test the uplink throughput as follows.

In the UE container:
```shell
iperf -c <ip_of_ext_dn> -B <ip_of_tunnel_interface> -u -i 1 -t 20 -b <amount_of_data_to_send> # you might need to play around with the -b option a bit
```

In the `oai-ext-dn` container:
```shell
iperf -s -u -i 1
```

**Question**: List the commands that you ended up running, and give the average uplink throughput. Does this value align with your expectations, and why (not)?

**Answer**: <span style="color:red">**TODO**</span>

---
<span style="color:red">**IMPORTANT**</span>: Once you are done with this lab, make sure to save a copy of your configuration files, since you will most likely change some of your configurations in the next lab.

In the end, make sure you have one archive named `lab4-oai.tar.gz` in the `traces` directory, containing your configuration files for this lab. For simplicity, you can archive the whole `oai` directory, **but make sure that the configuration files reflect the changes you performed in this lab!**